In [4]:
import pandas as pd
from hmmlearn.hmm import GaussianHMM
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools

In [5]:



path = "../../dataChallenge/"
name = "Scenario03-01/groundTruth.txt"

data = pd.read_csv(path+name, sep = " ", header = None)
print(len(data))
data.head()     

7722


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,1,2,1,286,178,290,208,268,208,264,...,318,376,282,174,290,174,276,178,298,178
1,1,3,0,414,150,452,182,438,182,438,...,-1,-1,416,146,418,144,-1,-1,442,142
2,1,1,0,188,152,152,190,134,190,126,...,202,426,182,146,-1,-1,166,148,-1,-1
3,2,2,1,286,178,290,208,268,208,264,...,318,376,282,174,290,174,276,178,298,178
4,2,3,0,414,152,450,180,436,182,438,...,-1,-1,-1,-1,418,144,-1,-1,440,142


In [7]:
nr_of_people = data.iloc[:,1].max()
data_list = [data[data.iloc[:,1]==i+1] for i in xrange(nr_of_people)]
print(data_list)


[        0   1   2    3    4    5    6    7    8    9  ...    29   30   31  \
2        1   1   0  188  152  152  190  134  190  126 ...   202  426  182   
5        2   1   0  188  152  152  190  134  190  126 ...   202  426  182   
8        3   1   0  188  152  152  190  134  190  126 ...   202  426  182   
11       4   1   0  188  152  152  190  134  190  126 ...   200  426  182   
14       5   1   0  188  152  152  190  134  190  126 ...   202  426  182   
17       6   1   0  186  152  152  190  134  190  124 ...   202  426  182   
20       7   1   0  186  152  150  190  134  190  124 ...   200  426  182   
23       8   1   0  184  154  150  190  132  190  122 ...   202  428  180   
26       9   1   0  184  152  150  190  132  190  122 ...   200  428  180   
29      10   1   0  182  152  148  190  130  190  120 ...   200  428  178   
32      11   1   0  180  152  148  190  130  190  120 ...   198  428  176   
35      12   1   0  180  152  146  190  128  190  118 ...   198  426  176  

In [36]:
model = GaussianHMM(n_components = 2)

for d in data_list:
    y = np.array(d.iloc[:,2])
    #print(y.shape)
    x = np.array(d.iloc[:,3:])
    #print(x.shape)
    model.fit(x)
    print("transition probabilities: \n {}".format(model.transmat_))
    pred = model.predict(x)
    cm = confusion_matrix(y, pred)
    norm_cm = cm.astype(float)
    for i, row in enumerate(cm.astype(float)):
        norm_cm[i,:] = row/np.sum(row)
    spr = float(cm[0,0]*cm[1,1]-cm[0,1]*cm[1,0])**2/(cm[0,0]**2+cm[0,1]**2)/(cm[1,0]**2+cm[1,1]**2)
    print(norm_cm)
    print("sin",spr**.5)
    


transition probabilities: 
 [[  9.99562172e-001   4.37828374e-004]
 [  9.16927636e-210   1.00000000e+000]]
[[ 0.90351873  0.09648127]
 [ 0.85221675  0.14778325]]
('sin', 0.065275650632228455)
transition probabilities: 
 [[ 0.99683258  0.00316742]
 [ 0.01675978  0.98324022]]
[[ 0.83530678  0.16469322]
 [ 0.87439024  0.12560976]]
('sin', 0.051966651006738834)
transition probabilities: 
 [[ 0.99736822  0.00263178]
 [ 0.00283522  0.99716478]]
[[ 0.57687012  0.42312988]
 [ 0.6575      0.3425    ]]
('sin', 0.1520232106695163)


In [22]:
def devide_into_frames(data):
    frames = []
    frame_count = data.iloc[:,0].max()
    for i in xrange(frame_count):
        frames.append(data[data.iloc[:,0]==i+1])
    return frames
    

frames = devide_into_frames(data)
print("length: {}".format(len(frames)-1))
#print(frames[10])

def identify_people(frames):
    hist = []
    prev = [None,None,None]
    for i, frame in enumerate(frames):
        if len(frame) != 3:
            continue
        def dist(A, B):
            if A is None or B is None: return np.inf
            s = 0.0
            n = 0
            for Ai, Bi in zip(A, B):
                if Ai == -1 or Bi == -1:
                    continue
                s += (Ai - Bi)**2
                n += 1
            return s/n if n != 0 else np.inf    
            #return np.linalg.norm((A-B)[1:])
        def score(perm):
            now = [frame[i] for i in perm]
            return sum(dist(previ, nowi) for previ, nowi in zip(prev, now))
        
        tmp = [score(i) for i in itertools.permutations(range(3))]
        print(tmp)
        
        perm = min(itertools.permutations(range(3)), key=score)
        hist.append(perm)
        prev = [frame[i] for i in perm]
    return hist
    
ret = identify_people(frames)

length: 2573
[inf, inf, inf, inf, inf, inf]
[1.0, 8.333333333333332, 2.3333333333333335, 8.333333333333332, 5.0, 3.6666666666666665]
[2.333333333333333, 7.0, 3.0, 11.0, 7.666666666666667, 11.0]
[1.0, 8.333333333333332, 6.333333333333334, 19.0, 15.666666666666666, 21.0]
[1.0, 8.333333333333332, 14.333333333333332, 30.333333333333336, 27.0, 35.666666666666664]
[1.0, 8.333333333333332, 26.333333333333336, 45.66666666666667, 42.333333333333336, 54.333333333333336]
[1.0, 8.333333333333332, 42.333333333333336, 65.0, 61.666666666666664, 77.0]
[1.0, 8.333333333333332, 62.33333333333333, 88.33333333333334, 85.0, 103.66666666666666]
[1.0, 8.333333333333332, 86.33333333333333, 115.66666666666666, 112.33333333333333, 134.33333333333331]
[2.333333333333333, 7.0, 115.00000000000001, 146.33333333333334, 143.0, 169.66666666666669]
[2.333333333333333, 7.0, 147.0, 181.66666666666669, 178.33333333333334, 208.33333333333334]
[1.0, 8.333333333333332, 182.33333333333334, 221.66666666666669, 218.333333333333

[1.0, 9.666666666666666, 29042.333333333332, 29452.333333333336, 29448.999999999996, 29850.333333333336]
[1.0, 9.666666666666666, 29526.333333333332, 29939.666666666664, 29936.333333333332, 30341.0]
[1.0, 9.666666666666666, 30014.333333333332, 30431.0, 30427.666666666664, 30835.666666666664]
[1.0, 9.666666666666666, 30506.333333333332, 30926.333333333336, 30922.999999999996, 31334.333333333336]
[1.0, 9.666666666666666, 31002.333333333332, 31425.666666666664, 31422.333333333332, 31837.0]
[1.0, 9.666666666666666, 31502.333333333332, 31929.0, 31925.666666666664, 32343.666666666664]
[1.0, 9.666666666666666, 32006.333333333332, 32436.333333333332, 32432.999999999996, 32854.33333333333]
[1.0, 9.666666666666666, 32514.333333333336, 32947.66666666667, 32944.333333333336, 33369.0]
[1.0, 9.666666666666666, 33026.333333333336, 33463.0, 33459.66666666667, 33887.66666666667]
[1.0, 9.666666666666666, 33542.333333333336, 33982.33333333333, 33979.00000000001, 34410.33333333333]
[1.0, 9.666666666666666

[1.0, 8.333333333333332, 89466.33333333333, 90179.0, 90175.66666666667, 90881.0]
[2.333333333333333, 7.0, 90315.0, 91029.66666666666, 91026.33333333333, 91736.33333333333]
[2.333333333333333, 9.666666666666666, 91167.0, 91886.33333333334, 91882.99999999999, 92595.0]
[1.0, 8.333333333333332, 92022.33333333333, 92745.0, 92741.66666666667, 93457.0]
[1.0, 8.333333333333332, 92882.33333333333, 93608.33333333333, 93605.0, 94323.66666666666]
[1.0, 8.333333333333332, 93746.33333333333, 94475.66666666666, 94472.33333333333, 95194.33333333334]
[1.0, 8.333333333333332, 94614.33333333333, 95347.0, 95343.66666666667, 96069.0]
[1.0, 8.333333333333332, 95486.33333333333, 96222.33333333333, 96219.0, 96947.66666666666]
[1.0, 8.333333333333332, 96362.33333333333, 97101.66666666666, 97098.33333333333, 97830.33333333334]
[1.3333333333333333, 8.0, 97242.66666666666, 97984.66666666666, 97834.0, 98569.33333333333]
[1.0, 7.666666666666668, 98126.33333333333, 98723.0, 98720.33333333333, 99310.33333333334]
[1.0

[1.0, 5.0, 184226.33333333334, 185040.33333333334, 185037.6666666667, 185847.6666666667]
[1.0, 5.0, 185442.33333333334, 186259.0, 186256.33333333334, 187069.0]
[1.0, 5.0, 186662.33333333334, 187481.6666666667, 187479.0, 188294.3333333333]
[1.0, 5.0, 187886.33333333334, 188708.33333333334, 188705.6666666667, 189523.6666666667]
[1.0, 5.0, 189114.33333333334, 189939.0, 189936.33333333334, 190757.0]
[1.0, 5.0, 190346.33333333334, 191173.6666666667, 191171.0, 191994.3333333333]
[1.0, 5.0, 191582.33333333334, 192412.33333333334, 192409.6666666667, 193235.6666666667]
[1.0, 5.0, 192822.33333333334, 193655.0, 193652.33333333334, 194481.0]
[1.0, 5.0, 194066.33333333334, 194901.6666666667, 194899.0, 195730.3333333333]
[1.0, 5.0, 195314.33333333334, 196152.33333333334, 196149.6666666667, 196983.6666666667]
[1.0, 5.0, 196566.33333333334, 197407.0, 197404.33333333334, 198241.0]
[1.0, 5.0, 197822.33333333334, 198665.6666666667, 198663.0, 199502.3333333333]
[1.0, 5.0, 199082.33333333334, 199928.333333

[2.333333333333333, 9.666666666666666, 363807.0, 365236.3333333334, 365232.99999999994, 366655.0]
[2.333333333333333, 9.666666666666666, 365515.0, 366947.6666666666, 366944.3333333333, 368369.6666666666]
[1.0, 8.333333333333332, 367226.3333333333, 368662.3333333333, 368659.0, 370087.6666666666]
[1.0, 8.333333333333332, 368942.3333333333, 370381.6666666666, 370378.3333333333, 371810.3333333334]
[2.333333333333333, 9.666666666666666, 370663.0, 372105.6666666666, 372102.3333333333, 373537.6666666666]
[1.0, 8.333333333333332, 372386.3333333333, 373832.3333333333, 373829.0, 375267.6666666666]
[1.0, 8.333333333333332, 374114.3333333333, 375563.6666666666, 375560.3333333333, 377002.3333333334]
[1.0, 8.333333333333332, 375846.3333333333, 377299.0, 377295.6666666667, 378741.0]
[1.0, 8.333333333333332, 377582.3333333333, 379038.3333333333, 379035.0, 380483.6666666666]
[1.0, 8.333333333333332, 379322.3333333333, 380781.6666666666, 380778.3333333333, 382230.3333333334]
[1.0, 8.333333333333332, 381

...

[1.0, 8.333333333333332, 8883114.333333334, 8890147.0, 8890143.666666666, 8897169.0]
[1.0, 8.333333333333332, 8891546.333333334, 8898582.333333334, 8898579.0, 8905607.666666668]
[1.0, 8.333333333333332, 8899982.333333334, 8907021.666666668, 8907018.333333334, 8914050.333333332]
[1.0, 8.333333333333332, 8908422.333333334, 8915465.0, 8915461.666666666, 8922497.0]
[1.0, 8.333333333333332, 8916866.333333334, 8923912.333333334, 8923909.0, 8930947.666666668]
[1.0, 8.333333333333332, 8925314.333333334, 8932363.666666668, 8932360.333333334, 8939402.333333332]
[1.0, 8.333333333333332, 8933766.333333334, 8940819.0, 8940815.666666666, 8947861.0]
[1.0, 8.333333333333332, 8942222.333333334, 8949278.333333334, 8949275.0, 8956323.666666668]
[1.0, 8.333333333333332, 8950682.333333334, 8957741.666666668, 8957738.333333334, 8964790.333333332]
[1.0, 8.333333333333332, 8959146.333333334, 8966209.0, 8966205.666666666, 8973261.0]
[1.0, 8.333333333333332, 8967614.333333334, 8974680.333333334, 8974677.0, 8981

[1.0, 7.666666666666668, 9702014.333333334, 9707893.666666668, 9707891.000000002, 9713763.666666668]
[1.0, 7.666666666666668, 9710826.333333334, 9716708.333333332, 9716705.666666668, 9722581.0]
[1.0, 7.666666666666668, 9719642.333333334, 9725527.0, 9725524.333333334, 9731402.333333332]
[1.0, 7.666666666666668, 9728462.333333334, 9734349.666666668, 9734347.000000002, 9740227.666666668]
[1.0, 7.666666666666668, 9737286.333333334, 9743176.333333332, 9743173.666666668, 9749057.0]
[1.0, 7.666666666666668, 9746114.333333334, 9752007.0, 9752004.333333334, 9757890.333333332]
[1.0, 7.666666666666668, 9754946.333333334, 9760841.666666668, 9760839.000000002, 9766727.666666668]
[1.3333333333333333, 8.0, 9763782.666666668, 9769680.666666668, 9771151.333333334, 9777042.666666666]
[1.0, 8.333333333333332, 9772622.333333334, 9779998.333333334, 9779995.0, 9787363.666666668]
[1.0, 8.333333333333332, 9781466.333333334, 9788845.666666668, 9788842.333333334, 9796214.333333332]
[1.0, 8.333333333333332, 9790

[1.0, 4.333333333333334, 10788014.333333334, 10792662.333333334, 10792660.333333334, 10797305.0]
[3.6666666666666665, 4.333333333333334, 10797306.333333334, 10801956.333333334, 10801954.333333334, 10806603.666666668]
[3.6666666666666665, 4.333333333333334, 10806602.333333334, 10811254.333333334, 10811252.333333334, 10815903.666666668]
[1.6666666666666665, 4.333333333333334, 10815902.333333334, 10820556.333333334, 10820554.333333334, 10825205.666666668]
[1.6666666666666665, 4.333333333333334, 10825206.333333334, 10829862.333333334, 10829860.333333334, 10834513.666666668]
[1.6666666666666665, 4.333333333333334, 10834514.333333334, 10839172.333333334, 10839170.333333334, 10843825.666666668]
[1.0, 4.333333333333334, 10843826.333333334, 10848486.333333334, 10848484.333333334, 10853141.0]
[1.6666666666666665, 4.333333333333334, 10853142.333333334, 10857804.333333334, 10857802.333333334, 10862461.666666668]
[3.6666666666666665, 4.333333333333334, 10862462.333333334, 10867126.333333334, 108671

[1.0, 7.666666666666668, 11582486.333333334, 11588909.666666668, 11588907.000000002, 11595323.666666668]
[1.6666666666666665, 7.666666666666668, 11592114.333333334, 11598540.333333332, 11598537.666666668, 11604957.666666668]
[1.6666666666666665, 7.666666666666668, 11601746.333333334, 11608175.0, 11608172.333333334, 11614595.0]
[1.9999999999999998, 8.666666666666666, 11611382.666666668, 11617814.666666668, 11619418.000000002, 11625843.333333334]
[1.0, 8.333333333333332, 11621022.333333334, 11629065.0, 11629061.666666666, 11637097.0]
[1.0, 8.333333333333332, 11630666.333333334, 11638712.333333334, 11638709.0, 11646747.666666668]
[1.0, 8.333333333333332, 11640314.333333334, 11648363.666666668, 11648360.333333334, 11656402.333333332]
[1.0, 8.333333333333332, 11649966.333333334, 11658019.0, 11658015.666666666, 11666061.0]
[1.3333333333333333, 6.666666666666666, 11659622.666666668, 11667676.666666668, 11666064.666666666, 11674113.333333332]
[1.0, 5.0, 11669282.333333334, 11675728.333333334, 

[1.0, 7.666666666666668, 12445062.333333334, 12451720.333333332, 12451717.666666668, 12458369.0]
[1.3333333333333333, 8.0, 12455042.666666668, 12461703.333333334, 12463364.666666668, 12470018.666666668]
[2.333333333333333, 7.0, 12465027.0, 12473355.0, 12473351.666666668, 12481675.0]
[1.0, 8.333333333333332, 12475014.333333334, 12483347.0, 12483343.666666666, 12491669.0]
[3.6666666666666665, 8.333333333333332, 12485006.333333334, 12493342.333333334, 12493339.0, 12501670.333333334]
[1.0, 8.333333333333332, 12495002.333333334, 12503341.666666668, 12503338.333333334, 12511670.333333332]
[2.333333333333333, 9.666666666666666, 12505003.0, 12513345.666666668, 12513342.333333334, 12521677.666666668]
[1.0, 8.333333333333332, 12515006.333333334, 12523352.333333334, 12523349.0, 12531687.666666668]
[1.0, 8.333333333333332, 12525014.333333334, 12533363.666666668, 12533360.333333334, 12541702.333333332]
[1.0, 8.333333333333332, 12535026.333333334, 12543379.0, 12543375.666666666, 12551721.0]
[1.0, 8.

In [23]:
ret

[(0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0, 1, 2),
 (0,

In [24]:
all(i==(0,1,2) for i in ret)

True

In [19]:
frames[10]

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
30,11,3,0,402,152,440,182,428,182,430,...,470,448,-1,-1,408,146,-1,-1,428,146
31,11,2,1,290,180,290,208,268,210,264,...,316,376,284,176,294,174,276,178,300,178
32,11,1,0,180,152,148,190,130,190,120,...,198,428,176,146,-1,-1,160,148,-1,-1
